##  Simulations of strain design

In [1]:
# in this section we simulate the different strain design that we have proposed before
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
from cameo import fba, pfba, flux_variability_analysis
from cameo.core import manipulation
from cameo.flux_analysis import structural

In [ ]:
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
plotter = PlotlyPlotter()

from cameo.strain_design.deterministic.flux_variability_based import FSEOF
from cameo.strain_design.deterministic import DifferentialFVA

In [2]:
# Verify working directory and change it if needed
import os
os.getcwd()
os.chdir("C:/Users/Deborah/anaconda3/envs/cellfactorydesign/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae/") #change accordingly

In [3]:
# Read the heterologous model
model = read_sbml_model("data/models/yeast8_resv_glc.xml")

In [16]:
model.metabolites.get_by_id('coumarate4')
model.reactions.get_by_id('HA-TAL')
model.metabolites.get_by_id('tyr__L_c')

Metabolite identifier,tyr__L_c
Name,L-tyrosine
Memory address,0x014bba415b08
Formula,C9H11NO3
Compartment,c
In 12 reaction(s),"TYRt2r, HA-TAL, TYRt6, TYRTA, TYRTRA, TYRPc, TYRt7, TYRTRS, TYR__Ltx, TYRNFT, r1078, TYR__Ltv"


#### Flux Balance Analysis of the heterologous model with the default settings

In [4]:
# reference model with maximized biomass production and default parameters
m_ref=model.copy()

In [10]:
fba1=fba(m_ref, objective=m_ref.reactions.GROWTH)
fba_ref=fba1.data_frame
fba_ref.sort_values(by='flux', ascending=False, inplace=True)
print('flux of oxygen uptake: ', fba_ref.loc['EX_o2_e'][0], 'unit')
print('flux of glucose uptake: ', fba_ref.loc['EX_glc__D_e'][0], 'unit')
print('flux of biomass production: ', fba_ref.loc['GROWTH'][0], 'unit')
print('flux of resveratrol production: ', fba_ref.loc['VVVST1'][0], 'unit')
fba_ref

flux of oxygen uptake:  -2.3315329209467555 unit
flux of glucose uptake:  -1.0 unit
flux of biomass production:  0.08192806489430403 unit
flux of resveratrol production:  0.0 unit


,flux
FERCOXOXI,9.037760
ATPtm,6.330327
ATPS3m,5.734898
PIt2m,5.162975
UBICRED,4.518880
...,...
EX_glc__D_e,-1.000000
CO2tm,-1.832126
EX_o2_e,-2.331533
H2Ot,-4.338799


#### Flux Balance Analysis of the heterologous model with increased glucose uptake

In [9]:
# new model copy for optimizing growth of yeast by increasing the glucose uptake
m_glu=model.copy()

In [13]:
print('Default glucose bounds: ', m_glu.reactions.EX_glc__D_e.bounds) #show default glucose uptake

manipulation.increase_flux(m_glu.reactions.EX_glc__D_e, -1.0, -10.0) # increasing the uptake of glucose from -1.0 to -10.0
print('New glucose bounds: ', m_glu.reactions.EX_glc__D_e.bounds) # check if the boundaries were updated

# flux balance analysis to see the effects on the oxygen uptake, growth and our product
fba2=fba(m_glu, objective=m_glu.reactions.GROWTH)
fba_glu=fba2.data_frame
fba_glu.sort_values(by='flux', ascending=False, inplace=True)
print('flux of oxygen uptake: ', fba_glu.loc['EX_o2_e'][0], 'unit')
print('flux of glucose uptake: ', fba_glu.loc['EX_glc__D_e'][0], 'unit')
print('flux of biomass production: ', fba_glu.loc['GROWTH'][0], 'unit')
print('flux of resveratrol production: ', fba_glu.loc['VVVST1'][0], 'unit')
fba_glu

Default glucose bounds:  (-10.0, -10.0)
New glucose bounds:  (-10.0, -10.0)
flux of oxygen uptake:  -22.065526992398787 unit
flux of glucose uptake:  -10.0 unit
flux of biomass production:  0.8471925464573381 unit
flux of resveratrol production:  0.0 unit


,flux
FERCOXOXI,85.280142
ATPtm,60.187375
ATPS3m,55.296138
PIt2m,48.116143
EX_h2o_e,42.822040
...,...
EX_glc__D_e,-10.000000
CO2tm,-18.654943
EX_o2_e,-22.065527
H2Ot,-42.822040


We can see that an increased uptake of glucose directly impacts the growth of the host linearly. The growth has increased from 0.082 to 0.84. The product flux is still zero. Therefore, we set the target now to the production of our product:

In [12]:
#### Flux Balance Analysis with increased glucose uptake and resveratrol production as objective

In [15]:
# copy of the model with the new objective of resveratrol
m_res=model.copy()

# increasing the uptake of glucose from -1.0 to -10.0
manipulation.increase_flux(m_res.reactions.EX_glc__D_e, -1.0, -10.0)

fba3=fba(m_res, objective=m_res.reactions.VVVST1)
fba_res=fba3.data_frame
fba_res.sort_values(by='flux', ascending=False, inplace=True)
print('flux of oxygen uptake: ', fba_res.loc['EX_o2_e'][0], 'unit')
print('flux of glucose uptake: ', fba_res.loc['EX_glc__D_e'][0], 'unit')
print('flux of biomass production: ', fba_res.loc['GROWTH'][0], 'unit')
print('flux of resveratrol production: ', fba_res.loc['VVVST1'][0], 'unit')
fba_res

flux of oxygen uptake:  -10.564224665058838 unit
flux of glucose uptake:  -10.0 unit
flux of biomass production:  0.0 unit
flux of resveratrol production:  3.189404860318783 unit


,flux
FERCOXOXI,42.256899
EX_h2o_e,40.863571
ATPtm,33.990997
ATPS3m,33.990997
UBICRED,21.128449
...,...
AKGCITtm,-8.769958
EX_glc__D_e,-10.000000
EX_o2_e,-10.564225
H2Otm,-38.972858


Even if we use the glucose of 10 [unit], we can see that the production of resveratrol leads to the death of the cell. Therefore, in the following model, we increase the growth of the host by higher glucose uptake and set a boundary for the growth of the host. The target of the model is still the resveratrol production.

#### Flux Balance Analysis with increased glucose uptake, minimum growth rate of yeast and resveratrol target

In [16]:
m_res_glu=model.copy()

# increasing the glucose uptake
manipulation.increase_flux(m_res_glu.reactions.EX_glc__D_e, -1.0, -10.0)

# setting the constraint of having a minimum growth of 0.3 [unit]
m_res_glu.reactions.GROWTH.bounds=(0.4, 1000)

# flux balance analysis to see the effects on the oxygen uptake, growth and our product
fba4=fba(m_res_glu, objective=m_res_glu.reactions.VVVST1)
fba_res_glu=fba4.data_frame
fba_res_glu.sort_values(by='flux', ascending=False, inplace=True)
print('flux of oxygen uptake: ', fba_res_glu.loc['EX_o2_e'][0], 'unit')
print('flux of glucose uptake: ', fba_res_glu.loc['EX_glc__D_e'][0], 'unit')
print('flux of biomass production: ', fba_res_glu.loc['GROWTH'][0], 'unit')
print('flux of resveratrol production: ', fba_res_glu.loc['VVVST1'][0], 'unit')
fba_res_glu

flux of oxygen uptake:  -15.821180645580405 unit
flux of glucose uptake:  -10.0 unit
flux of biomass production:  0.4 unit
flux of resveratrol production:  1.6947190788672892 unit


,flux
FERCOXOXI,61.876794
ATPtm,45.330542
ATPS3m,42.720620
EX_h2o_e,41.721151
UBICRED,30.938397
...,...
EX_glc__D_e,-10.000000
CO2tm,-10.135733
EX_o2_e,-15.821181
H2Ot,-41.721151


So, now the flux of the product is lower due to cell growth but the cells are still alive.

#### Parsimonious FBA for last cenario

In [19]:
print('Biomass bounds: ', m_res_glu.reactions.GROWTH.bounds)
print('Glucose bounds: ', m_res_glu.reactions.EX_glc__D_e.bounds)

pfba=pfba(m_res_glu, objective=m_res_glu.reactions.VVVST1)
pfba_res=pfba.data_frame
pfba_res.sort_values(by='flux', ascending=False, inplace=True)
print('flux of oxygen uptake: ', pfba_res.loc['EX_o2_e'][0], 'unit')
print('flux of glucose uptake: ', pfba_res.loc['EX_glc__D_e'][0], 'unit')
print('flux of biomass production: ', pfba_res.loc['GROWTH'][0], 'unit')
print('flux of resveratrol production: ', pfba_res.loc['VVVST1'][0], 'unit')
pfba_res

Biomass bounds:  (0.4, 1000)
Glucose bounds:  (-10.0, -10.0)
flux of oxygen uptake:  -15.821180645583794 unit
flux of glucose uptake:  -10.0 unit
flux of biomass production:  0.4 unit
flux of resveratrol production:  1.6947190788672892 unit


,flux
FERCOXOXI,61.876794
ATPtm,45.380305
ATPS3m,42.820147
EX_h2o_e,41.721151
UBICRED,30.938397
...,...
EX_glc__D_e,-10.000000
CO2tm,-10.285024
EX_o2_e,-15.821181
H2Ot,-41.721151


#### Flux Variabilty Analysis

In [26]:
%%time
fva_res = flux_variability_analysis(m_res_glu)
fva=fva_res.data_frame.sort_values(by='upper_bound', ascending=False)
print('lower bound of oxygen uptake: ', fva.loc['EX_o2_e'][0], '   and   upper bound of oxygen uptake: ', fva.loc['EX_o2_e'][1])
print('lower bound of glucose uptake: ', fva.loc['EX_glc__D_e'][0], '   and   upper bound of glucose uptake: ', fva.loc['EX_glc__D_e'][1])
print('lower bound of biomass production: ', fva.loc['GROWTH'][0], '   and   upper bound of biomass production: ', fva.loc['GROWTH'][1])
print('lower bound of resveratrol production: ', fva.loc['VVVST1'][0], '   and   upper bound of resveratrol production: ', fva.loc['VVVST1'][1])
fva

lower bound of oxygen uptake:  -42.08932636798315    and   upper bound of oxygen uptake:  -6.862714391742457
lower bound of glucose uptake:  -10.0    and   upper bound of glucose uptake:  -10.0
lower bound of biomass production:  0.4    and   upper bound of biomass production:  0.8471925464547586
lower bound of resveratrol production:  0.0    and   upper bound of resveratrol production:  1.6947190788696214
Wall time: 1min 48s


,lower_bound,upper_bound
HEXCCOAtrm,-1000.000000,1000.000000
ACONTa,-1000.000000,1000.000000
PC160181trmmm,-1000.000000,1000.000000
AKGt,-1000.000000,1000.000000
HDCEAtlp,-1000.000000,1000.000000
...,...,...
EX_nh4_e,-43.234315,-2.859822
EX_o2_e,-42.089326,-6.862714
EX_glc__D_e,-10.000000,-10.000000
H2Ot,-154.669139,-15.681733


#### Saving the flux data and the model as json files

In [27]:
import json
import xmltodict
os.chdir("C:/Users/Deborah/anaconda3/envs/cellfactorydesign/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae/data/models/")

with open('yeast8_resv_glc.xml') as xml_file:
    data_dict = xmltodict.parse(xml_file.read())
    json_data = json.dumps(data_dict)
with open("data.json", "w") as json_file:
        json_file.write(json_data)

In [36]:
import json
import xmltodict
os.chdir("C:/Users/Deborah/anaconda3/envs/cellfactorydesign/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae/data/models/")

json_fba=fba_res_glu.to_json(orient="records")
parsed=json.loads(json_fba)
file=json.dumps(parsed, indent=4)
with open("fba_res_glu.json", "w") as json_file:
        json_file.write(file)

### Differential Flux Variability Analysis

In [63]:
# Set the original model as reference to be compared with a set of models where n-reaction bounds are varied
reference_model = m_res_glu.copy()
biomass_rxn = reference_model.reactions.BIOMASS_SC5_notrace
target = reference_model.metabolites.trans_resv

In [64]:
diff_fva = DifferentialFVA(design_space_model=m_res_glu,
                           reference_model=reference_model,
                           objective=target,
                           variables=[biomass_rxn],
                           normalize_ranges_by=biomass_rxn,
                           points=10)

In [65]:
result = diff_fva.run(surface_only=True)
result

,lower_bound,upper_bound,gaps,normalized_gaps,biomass,production,KO,flux_reversal,suddenly_essential,free_flux,reaction,excluded
reaction,,,,,,,,,,,,
PGL,0.000371,0.000371,-0.802099,-2.784754,0.12794,0.716157,False,False,False,False,PGL,False
GND,0.000371,0.000371,-0.802099,-2.784754,0.12794,0.716157,False,False,False,False,GND,False
G6PDH2r,0.000000,0.000000,-0.801635,-2.784754,0.12794,0.716157,True,False,False,False,G6PDH2r,False
PDHm,0.209950,0.209950,-0.498439,-0.819831,0.12794,0.716157,False,False,False,False,PDHm,False
CSm,0.130346,0.130346,-0.398933,-0.819831,0.12794,0.716157,False,False,False,False,CSm,False
...,...,...,...,...,...,...,...,...,...,...,...,...
PYK,16.792033,16.792033,-0.929957,69.685565,0.12794,0.716157,False,False,False,False,PYK,False
GAPD,18.291976,18.291976,0.253771,80.310856,0.12794,0.716157,False,False,False,False,GAPD,False
PGK,-18.291976,-18.291976,0.253771,80.310856,0.12794,0.716157,False,False,False,False,PGK,False
